In [1]:
import torch, dgl
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from dgl.data import DGLDataset, CoraGraphDataset
import dgl.function as fn
import pandas as pd
from colorama import Fore
from urllib.request import urlretrieve

In [2]:
class SAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model.

    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """
    
    def __init__(self, in_feat:int, out_feat:int):
        super(SAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        """Forward computation

        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        """
        with g.local_scope():
            g.ndata['h'] = h
            # update_all is a message passing API.
            g.update_all(message_func = fn.copy_u('h', 'm'), 
                         reduce_func = fn.mean('m', 'h_N'))
            h_N = g.ndata['h_N']
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [3]:
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats)
        self.conv2 = SAGEConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [4]:
class KarateClubGraphDataset(DGLDataset):
    def __init__(self):
        super().__init__(name='karate_club')
        
    def __len__(self): return 1
    def __getitem__(self, i): return self.graph
    
    def process(self):
        #NOTE download data
        # urlretrieve('https://data.dgl.ai/tutorial/dataset/members.csv', './members.csv')
        # urlretrieve('https://data.dgl.ai/tutorial/dataset/interactions.csv', './interactions.csv')
        nodes_data = pd.read_csv('dataset/Karate_Club/members.csv')
        edges_data = pd.read_csv('dataset/Karate_Club/interactions.csv')
        n_nodes  = nodes_data.shape[0]
        
        node_labels  = torch.tensor(nodes_data['Club'].astype('category').cat.codes.to_list())
        node_features = torch.from_numpy(nodes_data['Age'].to_numpy()).reshape(n_nodes, 1)
        edge_features = torch.from_numpy(edges_data['Weight'].to_numpy())
        edges_src = torch.from_numpy(edges_data['Src'].to_numpy())
        edges_dst = torch.from_numpy(edges_data['Dst'].to_numpy())
        
        #NOTE create graph with nodes and edges feature
        self.graph = dgl.graph((edges_src, edges_dst), num_nodes=n_nodes)
        self.graph = dgl.to_bidirected(self.graph)      # convert to undirected
        self.graph.ndata['feat'] = node_features        #NOTE learnable
        self.graph.ndata['label'] = node_labels
        self.graph.ndata['node_features'] = nn.Parameter(torch.randn(self.graph.num_nodes(), 10))

        self.graph.edata['weight'] = edge_features

        #NOTE If your dataset is a node classification dataset, you will need to assign
        # masks indicating whether a node belongs to training, validation, and test set.

        n_train = int(n_nodes * 0.6)
        n_val  = int(n_nodes * 0.2)
        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        val_mask  = torch.zeros(n_nodes, dtype=torch.bool)
        test_mask  = torch.zeros(n_nodes, dtype=torch.bool)
        train_mask[:n_train] = True
        val_mask[n_train:n_train + n_val] = True
        test_mask[n_train + n_val:] = True
        self.graph.ndata['train_mask'] = train_mask
        self.graph.ndata['val_mask'] = val_mask
        self.graph.ndata['test_mask'] = test_mask

In [5]:
dataset = CoraGraphDataset(verbose=False)
g = dataset[0]

In [8]:
g.ndata['feat'].shape

torch.Size([34, 1])

In [7]:
dataset = KarateClubGraphDataset()
g = dataset[0]

In [9]:
num_classes = len(set(g.ndata['label'].tolist()))   #2
model = GraphSAGE(g.ndata['feat'].shape[1], 16, num_classes)

In [12]:
def train(g:DGLDataset, model:nn.Module, epochs:int):
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    all_logits = []
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat'].float()  #BUG https://github.com/dmlc/dgl/issues/2299
    labels = g.ndata['label']
    
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(1, epochs+1):
        # Forward
        logits = model(g, features)
        
        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # NOTE that we should only compute the losses of the nodes in the training set,
        # i.e. with train_mask 1.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        all_logits.append(logits.detach())

        if not e % 25:
            print('In epoch {:3d}, loss: {:.4f}, val acc: {:.4f} (best {:.4f}), test acc: {:.4f} (best {:.4f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

In [13]:
train(g, model, 200)

/home/agent/anaconda3/envs/graph/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In epoch  25, loss: 0.9192, val acc: 0.8333 (best 0.8333), test acc: 0.7500 (best 1.0000)
In epoch  50, loss: 0.5191, val acc: 0.1667 (best 0.8333), test acc: 0.0000 (best 1.0000)
In epoch  75, loss: 0.4959, val acc: 0.1667 (best 0.8333), test acc: 0.0000 (best 1.0000)
In epoch 100, loss: 0.4880, val acc: 0.1667 (best 0.8333), test acc: 0.0000 (best 1.0000)
In epoch 125, loss: 0.4845, val acc: 0.1667 (best 0.8333), test acc: 0.0000 (best 1.0000)
In epoch 150, loss: 0.4813, val acc: 0.1667 (best 0.8333), test acc: 0.0000 (best 1.0000)
In epoch 175, loss: 0.4789, val acc: 0.1667 (best 0.8333), test acc: 0.0000 (best 1.0000)
In epoch 200, loss: 0.4760, val acc: 0.1667 (best 0.8333), test acc: 0.0000 (best 1.0000)
